In [4]:
#Import Library
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from collections import Counter

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from keras.utils import pad_sequences
from keras import mixed_precision
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from nltk import FreqDist
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Dense, Embedding, Bidirectional
stop = stopwords.words('english')
nltk.download('sentiwordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [1]:
from google.colab import files
files= files.upload()

Saving review_clean.csv to review_clean.csv


In [5]:
df=pd.read_csv('review_clean.csv')
df.head()

,stopwords,punctuation,hastags,is_english,review
0,5,3,0,True,"['wife', 'needed', 'rain', 'jacket', 'last', '..."
1,2,1,0,True,"['true', 'size', 'small']"
2,6,4,0,True,"['nice', 'coat', 'sleeves', 'seems', 'long', '..."
3,11,2,0,True,"['bought', 'alaska', 'cruise', 'worked', 'work..."
4,3,9,0,True,"['definitely', 'first', 'rain', 'stayed', 'sec..."


Two metrics, such as polarity and subjectivity, will be taken into account in this research.For sentiment analysis tasks like opinion mining, customer feedback analysis, and social media sentiment analysis, polarity and subjectivity scores can offer insights about the general sentiment and tone of a piece of text.

In [ ]:
#calculate the number of word
df['word_count'] = df['review'].apply(lambda x: len(str(x).split(" ")))
df[['review','word_count']].head()

In [ ]:
from textblob import TextBlob

df['review']= df['review'].astype(str)

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity


df['polarity'] = df['review'].apply(pol)
df['subjectivity'] = df['review'].apply(sub)
df.head()

We will identify if the review is good, neutral or bad reviews using the polarity score

In [ ]:
from nltk.corpus import wordnet
# Additional positive and negative keywords
positive_keywords = ['excellent', 'great', 'amazing', 'nice','value','pretty','cheaper','waterproof', 'durable', 'lightweight', 'breathable', 'functional', 'stylish', 'comfortable','fantastic', 'wonderful', 'awesome', 'outstanding', 'superb', 'perfect','love','ideal','useful','valuable',"Windproof",
    "Breathable","Quick-drying","Versatile","Fashionable","Lightweight",'elastic''long',"Compact","Reliable","Sturdy","High-quality","Functional","Convenient","Well-made","Stylish",    "Love", "Great", "Nice", "Perfect", "Lightweight",
    "Comfortable", "Ideal fit", "Fantastic choice", "Quality", "Good value", "True size", "Fits perfectly", "Cute","Comfy", "Excellent", "Happy", "Loved", "Fits well", "Comfy", "Excellent", "Happy", "Loved", "Fits well", "Comfy", "Excellent", "Happy", "Loved", "Fits well", "Wonderful", "Durable",
                     "Great fit", "Comfortable", "Lightweight", "Quality", "Versatile",
    "Warm", "Love", "Perfect", "Awesome", "Beautiful", "Excellent service",
    "Worth buying", "Well made", "Nice", "Loved", "Bigger than expected",
    "Fits well", "About backcountry", "Worth the price", "Good gift","Fits well", "Just like picture", "Waterproof", "Repels water", "Versatile",
    "Fits tight bust", "Present", "Material started fray", "Repels water"]

negative_keywords = ['poor', 'bad', 'terrible', 'bother','weird','long','','disappointing', 'awful', 'horrible', 'disgusting', 'atrocious', 'abysmal','leaks', 'flimsy', 'uncomfortable', 'heavy', 'bulky', 'poorly made', 'not waterproof',"Leaks","Tears easily",
    "Heavy","Bulky","Uncomfortable","Poorly constructed","Not waterproof","Impractical","Limited mobility","Lacks ventilation","Cheaply made","Not durable","Ineffective","Difficult to pack","Unattractive design","Short sleeves", "Overheated", "Weird", "Not described",
    "Runs small", "Thin", "Not warm", "Noisy", "Missing liner", "Defective", "Crunchy plastic texture", "Ran big","Awkward fit", "Not pictured", "Not waterproof","Faulty zipper", "Not good for cold weather", "Received used product", "Odd fit", "Not waterproof",
                     "Overpriced", "Low water resistance", "Not great", "Thin", "Flimsy",
    "Small sizes", "Not really cold weather", "Cheap", "Misleading",
    "Not worth the price", "Not as advertised", "Low quality", "Terrible",
    "False advertisement", "Poor quality", "Not new", "It doesn't look like a North Face jacket",
    "Simple", "Worthy of price", "Not worth the money","Returned defective product", "Defective product", "Not water resistant",
    "Not worth the money", "Runs small", "Low quality", "Scam", "Not as advertised",
    "Low quality", "Warning", "Damage", "Cheap product", "Poor quality", "Cheap product",
    "Wors gd lightweight", "Scam alert", "Runs small", "Wrong shipment", "Cheap product",
    "Plastic zipper"]

# Function to get synonyms of a word using WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower())
    return synonyms

# Add synonyms of existing positive and negative keywords to the lists
for keyword in list(positive_keywords):
    positive_keywords.extend(get_synonyms(keyword))
for keyword in list(negative_keywords):
    negative_keywords.extend(get_synonyms(keyword))

# Convert lists to sets to remove duplicates
positive_keywords = list(set(positive_keywords))
negative_keywords = list(set(negative_keywords))

In [ ]:
def classify_review(text):
    # Use TextBlob to calculate sentiment polarity
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity

    # Define threshold for positive sentiment
    threshold = 0.1

    # Classify review as "good" if polarity is above threshold, otherwise "not good"
    if polarity > threshold:
        return "1"
    else:
        return "0"

In [ ]:
df['review'][1]

In [ ]:
# Example usage:
review = df['review'][1]
classification = classify_review(review)
print("Review classification:", classification)

In [ ]:
# Apply classify_review function to the 'review' column in the DataFrame
df['review_classification'] = df['review'].apply(classify_review)


In [ ]:
df.head()

In [ ]:
#Round the polarity & subjectivity score
df['polarity'] = df['polarity'].round(2)
df['subjectivity'] = df['subjectivity'].round(2)
df.sample(5)

Below, we will visualize sentiment analysis results, where each point represents a data point with polarity and subjectivity values, and the index labels provide additional information about each point.

In [ ]:
pip install afinn

In [ ]:
import pandas as pd
from afinn import Afinn

# Initialize AFINN sentiment analyzer
afinn = Afinn()

# Function to calculate sentiment score for a given text
def calculate_sentiment(text):
    return afinn.score(text)

# Apply the sentiment analysis function to each review in the DataFrame
df['SentimentScore'] = df['review'].apply(calculate_sentiment)

# Classify sentiment based on score (positive if score > 0, negative if score < 0)
def classify_sentiment(score):
    if score > 0:
        return '1'
    else:
        return '0'
df['Sentiment'] = df['SentimentScore'].apply(classify_sentiment)


In [ ]:
df.head()

**Distribution of Review**

In [ ]:
a = df.groupby(['Sentiment'])['review'].count()
a = a.reset_index()
a = a.rename(columns={'Review': 'number of customer'})
fig, ax = plt.subplots(figsize=(4, 5))

sns.barplot(x='Sentiment', y='review', data=a, hue='Sentiment')

It can be seen that the class is unbalanced and we will handle it before modeling

1. Polarity

The text's sentiment orientation is measured by polarity, which indicates whether the attitude is positive, negative, or neutral. Usually, it is shown as a number scale, where:
Positive polarity indicates a positive sentiment.
Negative polarity indicates a negative sentiment.
Neutral polarity indicates a lack of sentiment or a neutral sentiment. Polarity range from -1 to 1.

In [ ]:
#Examining the polarity
num_bins = 50
plt.figure(figsize=(6,6))
n, bins, patches = plt.hist(df.polarity, num_bins, facecolor='blue', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Number of Reviews')
plt.title('Histogram of Polarity Score')
plt.show()


In [ ]:
plt.figure(figsize=(5,5))
sns.boxenplot(x='Sentiment', y='polarity', data=df,hue='Sentiment')
plt.show();

As this box plot illustrates,
- Some positive reviews have low polarity
- most of negative reviews have low polarity



In [ ]:
df.loc[(df.polarity == 1 & (df.Sentiment == 0))].review.head(10).tolist()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxenplot(x='punctuation', y='polarity', hue='Sentiment', data=df)
plt.xlabel('Punctuation', fontsize=13)
plt.ylabel('Polarity Score', fontsize=13)
plt.title('Punctuationr vs Polarity', fontsize=15)
plt.show()

Interpretation:

Median: The line within each box represents the median sentiment polarity score for reviews in that category of punctuation usage. A higher median indicates a more positive sentiment, while a lower median indicates a more negative sentiment.

Box Height: The height of the box represents the interquartile range (IQR), which indicates the spread of sentiment polarity scores within each category. A taller box suggests a wider range of sentiment expressions.

Whiskers: The whiskers extending from the boxes show the range of sentiment polarity scores within each category, excluding outliers. Outliers, if present, are usually represented as individual data points beyond the whiskers.

Outliers: Individual data points beyond the whiskers represent outlier reviews that have extreme sentiment polarity scores compared to the rest of the data within that category of punctuation usage. These outliers could be particularly positive or negative reviews.

In [ ]:
df_sub= df.loc[df.word_count //50 * 50]

In [ ]:
plt.figure(figsize=(6,5))
sns.barplot(x='word_count', y='polarity', hue='Sentiment', data=df_sub)
plt.xlabel('Number of Words', fontsize=13)
plt.ylabel('Polarity Score', fontsize=13)
plt.title('Number of Words vs Polarity', fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxenplot(x='word_count', y='polarity', hue='Sentiment',  linewidth=0.5, data=df)
plt.xlabel('Number of Words', fontsize=10)
plt.ylabel('Polarity Score', fontsize=10)
plt.title('Number of Words vs Polarity', fontsize=15)
plt.xlim([20, 100])
plt.xticks(rotation = 45)
plt.xticks(fontsize=8)
plt.show()


It can be seen that positive review have more words compare to bad review

2. Subjectivity

Subjectivity measures the degree to which the text expresses opinions, feelings, or beliefs, as opposed to factual information.The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [ ]:

# Density Plot and Histogram of subjectivity
plt.figure(figsize=(10,5))
sns.distplot(df['subjectivity'], hist=True, kde=True,
             bins=int(30), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
plt.xlim([-0.001,1.001])
plt.xlabel('Subjectivity', fontsize=13)
plt.ylabel('Frequency', fontsize=13)
plt.title('Distribution of Subjectivity Score', fontsize=15)

In [ ]:

#plt.figure(figsize=(10,6))
sns.relplot(x="subjectivity", y="polarity", hue="Sentiment", alpha=1 , palette="muted",
            height=6, aspect= 1.5, data=df)
plt.xlabel('Subjectivity', fontsize=13)
plt.ylabel('Polarity', fontsize=13)
plt.title('Subjectivity vs Polarity', fontsize=15)
plt.show();

In [ ]:
plt.figure(figsize=(10,6))
sns.boxenplot(x='Sentiment', y='subjectivity', data=df, hue='Sentiment')
plt.show()

Customer who gave positive review is more subjective than the one who give bad reviews

In [ ]:
plt.figure(figsize=(12,6))
df_sub= df.loc[df.word_count <=50]
sns.boxenplot(x='word_count', y='subjectivity', hue='Sentiment', data=df_sub, palette = 'Paired')
plt.xlabel('Number of Words', fontsize=13)
plt.ylabel('subjectivity Score', fontsize=13)
plt.title('Number of Words vs subjectivity', fontsize=13)
plt.show()

It can be observed that customer who gave good review is more subjective

Top 10 reviews that have highest polarity ( positive sentiment), 'good review' , and most subjective review

In [ ]:
df.loc[(df.polarity <0) & (df.subjectivity ==1) & (df["Sentiment"] == '0') , 'review'].head(10).tolist()

In [ ]:
# Top 10 reviews that have low polarity (negative sentiment) but 'good review' value is 1, and subjective (opinion):
df.loc[(df.polarity <=0.25 ) & (df.subjectivity >=0.8) & (df["Sentiment"] == '1')  , "review"].head(10).tolist()

In [ ]:
# Top 10 reviews that have low polarity (negative sentiment) but 'good review' value is 1, and subjective (opinion):
df.loc[(df.polarity <=0.25 ) & (df.subjectivity ==0) & (df["Sentiment"] == '0')  , "review"].head(10).tolist()

In [ ]:
from collections import Counter

# Initialize lists to store results
top_words_positive = []
top_words_negative = []

# Print the first few entries of the 'review' column to verify its contents
print("First few entries of the 'review' column:")
print(df['review'].head())

# Function to calculate the top 10 most frequent words for a given sentiment class
def top_10_frequent_words_for_sentiment(sentiment, tokenized_reviews):
    # Filter the tokenized reviews by sentiment
    filtered_reviews = [review for review, sent in zip(tokenized_reviews, df['Sentiment']) if sent == sentiment]

    # Print the number of reviews for the current sentiment
    print("Number of reviews for sentiment", sentiment, ":", len(filtered_reviews))

    # Flatten the list of tokens
    flattened_tokens = [token for sublist in filtered_reviews for token in sublist]

    # Count the occurrences of each token
    token_counts = Counter(flattened_tokens)

    # Get the top 10 most frequent words
    top_10_frequent_words = token_counts.most_common(10)

    return top_10_frequent_words

# Calculate the top 10 most frequent words for positive sentiment (sentiment 1)
top_10_frequent_words_1 = top_10_frequent_words_for_sentiment('1', df['review'])
for word, frequency in top_10_frequent_words_1:
    top_words_positive.append((word, frequency))

# Calculate the top 10 most frequent words for negative sentiment (sentiment 0)
top_10_frequent_words_0 = top_10_frequent_words_for_sentiment('0', df['review'])
for word, frequency in top_10_frequent_words_0:
    top_words_negative.append((word, frequency))

# Print or return the lists
print("Top 10 most frequent words for positive sentiment:")
print(top_words_positive)
print("\nTop 10 most frequent words for negative sentiment:")
print(top_words_negative)


In [ ]:
#authorisasi
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Save the data
df.to_csv('/content/drive/MyDrive/amazon/review_fix.csv', index=False)